In [25]:
import IPython
import numpy as np
from nputil import wav_read, chunk, zero_pad_to_n

In [36]:
wav, fs = wav_read('100039_sazgreen.wav')
dt = 1.0 / fs
wav = np.mean(wav, axis=1)
wavlen = wav.shape[0]
IPython.display.Audio(wav, rate=fs)

In [37]:
nfft = 1024
nhop = 512

wav_chunks = chunk(wav, nfft, hop=nhop)
#wav_chunks *= np.hanning(nfft)
nchunks = wav_chunks.shape[0]

spec_chunks = np.fft.rfft(wav_chunks, n=nfft)
spec_chunks_m = np.absolute(spec_chunks)
spec_chunks_p = np.angle(spec_chunks)

In [38]:
ncomponents = 8

from sklearn.decomposition import PCA

feats = np.stack([spec_chunks_m, spec_chunks_p], axis=1)
feats = np.reshape(feats, (feats.shape[0], -1))
print feats.shape

pca_f = PCA(n_components=ncomponents)
feats_xf = pca_f.fit_transform(feats)
print feats_xf.shape

pca_t = PCA(n_components=ncomponents)
feats_xft = pca_t.fit_transform(np.swapaxes(feats_xf, 0, 1))
print feats_xft.shape

(431L, 1026L)
(431L, 8L)
(8L, 8L)


In [39]:
recons_xf = np.swapaxes(pca_t.inverse_transform(feats_xft), 0, 1)
print recons_xf.shape
recons = pca_f.inverse_transform(recons_xf)
print recons.shape

recons_m = recons[:, :nfft // 2 + 1]
recons_p = recons[:, nfft // 2 + 1:]
recons_spec_chunks = recons_m * np.exp(1j * recons_p)
print recons_spec_chunks.shape

(431L, 8L)
(431L, 1026L)
(431L, 513L)


In [41]:
recons_wav = np.zeros(wavlen + nfft)
for i, recons_chunk in enumerate(recons_spec_chunks):
    chunk_wav = np.fft.irfft(recons_chunk)
    start = i * nhop
    recons_wav[start:start+nfft] = chunk_wav
IPython.display.Audio(recons_wav, rate=fs)